In [1]:
#Se instalan los paquetes de la libreria openpyxl
!pip install openpyxl

In [2]:
#Importamos las librerias pandas y numpy
import pandas as pd
import numpy as np 

In [3]:
#Se guardan en tablas los datasheet de los cluster oro, plata y bronce obtenidos anteriormente así como las categorias de los productos
data1 = pd.read_excel("Relacion Bronce.xlsx")
data2 = pd.read_excel("Relacion Plata.xlsx")
data3 = pd.read_excel("Relacion Oro.xlsx")
categorias=pd.read_excel("categorias_productos.xlsx")

#Se guardan en una lista las columnas que se van a descartar
Columns_To_Drop = ["MEMBRESIA_A", "NOMBRE_A", "CATEGORIA_A", "MEMBRESIA_B","NOMBRE_B","CATEGORIA_B"] 

#Se descartan las columnas en las tablas oro, plata y bronce y las convertimos en matrices de frecuencia
df1=data1.drop(Columns_To_Drop,axis=1)
tabla_frecuencias1=pd.crosstab(df1.ID_PRODUCTO_A,df1.ID_PRODUCTO_B,df1.CUANTOS,aggfunc=sum).fillna(0)
    
df2=data2.drop(Columns_To_Drop,axis=1)
tabla_frecuencias2=pd.crosstab(df2.ID_PRODUCTO_A,df2.ID_PRODUCTO_B,df2.CUANTOS,aggfunc=sum).fillna(0)

df3=data3.drop(Columns_To_Drop,axis=1)
tabla_frecuencias3=pd.crosstab(df3.ID_PRODUCTO_A,df3.ID_PRODUCTO_B,df3.CUANTOS,aggfunc=sum).fillna(0)

C:\Users\Erika Generador\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
#Función para obtener la categoria de un producto usando la tabla "categorias"
cat1=''
cat2=''
id_p1=''
id_p2=''

def obtener_categoria(cat1,id_p1):
    #Se obtiene la categoria del producto id_p1 que se ingrese 
    cat1=categorias.loc[categorias['PRO_ID_PRODUCTO']==id_p1].filter(items=['CAT_CATEGORIA']).values
    cat1=cat1.tolist()
    cat1=cat1[0][0]
    return cat1

In [22]:
#Se selecciona el grupo (oro, plata o bronce) al que pertenece el tendero
categorys= int(input("introduzca su categoria: "))

#Al elegir la opción, se iguala la matriz de frecuencias del grupo correspondiente
if categorys==1:
    print("Cliente Bronce")
    data = data1
    tabla2=tabla_frecuencias1.max()
    tabla_frecuencias=tabla_frecuencias1
    
elif categorys==2:
    print("Cliente plata")
    data = data2
    tabla2=tabla_frecuencias2.max()
    tabla_frecuencias=tabla_frecuencias2
    
elif categorys==3:
    print("Cliente oro")
    data = data3
    tabla2=tabla_frecuencias3.max()
    tabla_frecuencias=tabla_frecuencias3

introduzca su categoria: 1
Cliente Bronce


In [23]:
#Se cambia el tipo de dato de membresia_a a str
data["MEMBRESIA_A"] =  data["MEMBRESIA_A"].astype(str)
data["cate"] = data["MEMBRESIA_A"].str[:1]
data.head(2)

,ID_PRODUCTO_A,MEMBRESIA_A,NOMBRE_A,CATEGORIA_A,ID_PRODUCTO_B,MEMBRESIA_B,NOMBRE_B,CATEGORIA_B,CUANTOS,cate,Categoria
0,100139,2,Mundet 250 ml Durazno No Retornable 12 Piezas,BEBIDAS,ALB100015,2,Maruchan chile piquin 64gr,ABARROTES,8,2,NaN
1,100043,2,Valle Frut Guayaba 600ml 6pz,BEBIDAS,2909,2,Coca Cola 1.35 lts No Retornable 12 Piezas,BEBIDAS,7,2,NaN


In [24]:
#Se agregará a la tabla data el grupo al que pertenecen los clientes
category = {"3":"Oro","1":"plata"}

In [25]:
data["Categoria"] = data["cate"].map(category) #identifica conforme a month el nombre del mes de la nueva coumna
data.head(3)

,ID_PRODUCTO_A,MEMBRESIA_A,NOMBRE_A,CATEGORIA_A,ID_PRODUCTO_B,MEMBRESIA_B,NOMBRE_B,CATEGORIA_B,CUANTOS,cate,Categoria
0,100139,2,Mundet 250 ml Durazno No Retornable 12 Piezas,BEBIDAS,ALB100015,2,Maruchan chile piquin 64gr,ABARROTES,8,2,NaN
1,100043,2,Valle Frut Guayaba 600ml 6pz,BEBIDAS,2909,2,Coca Cola 1.35 lts No Retornable 12 Piezas,BEBIDAS,7,2,NaN
2,100019,2,SC Fresa+Vainilla 200ml Tetrapack No Ret 8 Pzs,LACTEOS,2909,2,Coca Cola 1.35 lts No Retornable 12 Piezas,BEBIDAS,7,2,NaN


In [26]:
#Se cambia el valor de membresia_a a tipo numerico
data["MEMBRESIA_A"] = pd.to_numeric(data["MEMBRESIA_A"])
data["cate"] = pd.to_numeric(data["cate"])

In [27]:
#Funciones para algoritmo caso n productos
#----------Verificar si productos delta no se encuentran en lista productos carrito
def inicio_caso_n():
    #Listas auxiliares para almacenar datos temporalmente
    Repetidos = [] #Lista repetidos
    Repetidos_aux=[] #Lista repetidos auxiliar
    ProductosA=[] #Lista productos carrito
    ProductosB=[] #Lista productos alpha
    
    #Listar id productos del carrito de compras
    for x in range(len(productos)):
        ProductosA.append(productos[x][0])
        ProductosB.append(productos[x][2])

    #Verificar si hay valores alpha en valores carrito y guardarlo su posición en repetidos_aux
    for x in range(len(ProductosA)):
        for y in range(len(ProductosB)):
            if x!=y:
                if ProductosB[y]==ProductosA[x] and ProductosB[y] not in Repetidos:
                    Repetidos_aux.append(y) #Guardamos la posiciónes repetidas con el principal

    #Eliminar duplicados de Repetidos_aux **Ver si hay mejor manera de hacer el for de arriba pagina guardada en favoritos
    for item in Repetidos_aux:
        if item not in Repetidos:
            Repetidos.append(item)

    #----------Decisión algoritmo
    if len(Repetidos)==0:
        print("IDs producto alpha no coinciden con IDs carrito -> verificar alphas repetidos")
        alphas_rep()
    
    else:
        #Eliminar productos alpha que coinciden con los productos del carrito
        for idx in sorted(Repetidos, reverse = True):
            del productos[idx]

        if len(productos)==1:
            #Recomendar producto
            print("Producto recomendado eliminar_alpha_carrito: ",productos[0][2])

        if len(productos)==0:
            print("Productos alpha ya estan en productos comprados en carrito-> Ir a recomendación extra")
            opciones_extra()

        else:
            print("Productos no repetidos alpha/carrito -> Ir a alphas repetidos")
            alphas_rep()
            
#Verificar si hay productos alpha repetidos
def alphas_rep():
    Repetidos=[]
    Repetidos_aux=[]
    lista_unicos=[]
    ProductosA=[] #Arreglo productos alpha

    #----------Obtener las veces que un producto alpha está repetido
    #Añadir a lista productos alpha
    for x in range(len(productos)):
        for y in range(1):
            ProductosA.append(productos[x][2])

    #Obtener valores unicos productos alpha
    for item in ProductosA:
        if item not in lista_unicos:
            lista_unicos.append(item)

    #Contar valores repetidos por cada producto alpha
    for i in range(len(lista_unicos)):
        Repetidos.append([lista_unicos[i],0])
        for j in range(len(ProductosA)):
            if lista_unicos[i]==ProductosA[j]:
                Repetidos[i][1]+=1

    #----------Obtener valor maximo y valores minimos
    #Verificar si hay un valor mayor o valores repetidos
    max=Repetidos[0][1]
    for x in range(len(Repetidos)):
        if Repetidos[x][1]>max:
            max=Repetidos[x][1]

    #obtener posiciones de todos los valores minimos al valor máximo
    min=[]
    for x in range(len(Repetidos)):
        if Repetidos[x][1]<max:
            min.append(Repetidos[x][0])

    #----------Eliminar los valores menores al valor máximo***
    #Obtener la posicion de los valores minimos a eliminar
    lugar=[]
    for i in range(len(productos)):
        for j in range(len(min)):
            if productos[i][2]==min[j]:
                lugar.append(i) 

    #Eliminar valores
    for idx in sorted(lugar, reverse = True):
        del productos[idx]

    #----------Verificar si los productos alpha son todos iguales **Arreglar este tambien
    productos_alpha_rep=[productos[0][2]]
    for i in range(len(productos)):
        if productos[i][2] not in productos_alpha_rep:
            productos_alpha_rep.append(productos[i][2])

    #----------Decisión algoritmo
    if len(productos_alpha_rep)==1:
        #Recomendar producto
        print("Producto recomendado alphas_rep: ",productos_alpha_rep)

    else:
        #Ir a frecuencia
        print(len(productos_alpha_rep),productos_alpha_rep,"Productos con mismas veces repetidas -> Ir a frecuencias")
        frecuencia(productos_alpha_rep)

#Verificar frecuencias mayor valores alpha grupos
def frecuencia(productos_alpha_rep: list):
    #----------Obtener frecuencia absoluta
    suma_frecuencias=[]
    for i in range(len(productos_alpha_rep)):
        suma_frecuencias.append([productos_alpha_rep[i],0])


    #Sumar frecuencias productos y realizar división*************
    for i in range(len(productos)):
        for j in range(len(suma_frecuencias)):
            if suma_frecuencias[j][0]==productos[i][2]:
                suma_frecuencias[j][1]=suma_frecuencias[j][1]+productos[i][3]

    #----------Obtener valor frecuencia maxima
    max=suma_frecuencias[0][1]
    pos=0
    for x in range(len(suma_frecuencias)):
        if suma_frecuencias[x][1]>max:
            max=suma_frecuencias[x][1]
            pos=x

    #Verificar si frecuencia máxima está duplicada
    frec_repetidos=[]
    conta=0
    for x in range(len(suma_frecuencias)):
        if suma_frecuencias[x][1]==max:
            frec_repetidos.append(suma_frecuencias[x][0])
            conta+=1

    productos_frec_rep=[]
    for i in range(len(frec_repetidos)):
        productos_frec_rep.append(frec_repetidos[i]) 
   
    #----------Decisión algoritmo
    if conta==1:
        #Recomendar producto
        print("Producto recomendado frecuencia: ",frec_repetidos)
        
    elif conta>1:
        #Ir a cantidad
        print(frec_repetidos,"Maximo frecuencia duplicado->Ir a cantidad")
        cantidad(productos_alpha_rep)

def cantidad(productos_alpha_rep: list):
    #Verificar cantidad mayor valores alpha grupos
    #----------Obtener suma de cantidades
    #Crear arreglo para suma de cantidad
    suma_cantidad=[]
    for i in range(len(productos_frec_rep)):
        suma_cantidad.append([productos_frec_rep[i],0])

    #Sumar frecuencias productos y realizar división*************
    for i in range(len(productos)):
        for j in range(len(suma_cantidad)):
            if suma_cantidad[j][0]==productos[i][2]:
                suma_cantidad[j][1]=suma_cantidad[j][1]+productos[i][1]

    #----------Obtener valor frecuencia maxima
    max=suma_cantidad[0][1]
    pos=0
    for x in range(len(suma_cantidad)):
        if suma_cantidad[x][1]>max:
            max=suma_cantidad[x][1]
            pos=x

    #Verificar si frecuencia máxima está duplicada
    cant_repetidos=[]
    conta=0
    for x in range(len(suma_cantidad)):
        for y in range(1):
            if suma_cantidad[x][1]==max:
                cant_repetidos.append(suma_cantidad[x][0])
                conta+=1

    #----------Decisión algoritmo
    if conta==1:
        #Recomendar ese producto
        print("Producto recomendado cantidad: ",cant_repetidos)
        
    elif conta>1:
        print(cant_repetidos,"Maxima cantidad duplicado->Ir a opciones extra")
        opciones_extra()

#Opciones extra
def opciones_extra():
    #Productos con frecuencia máxima
    #print("Productos recomendados: ",max_value_product_1,max_value_product_2)

    #Ingresar producto en promoción
    producto_promo=input("Ingresar producto en promoción: ")
    print("Producto recomendado: ", producto_promo)

    #Producto con mayor inventario

In [28]:
#Variables auxiliares
cat1=''
cat2=''
cat3=''
id_p1=''
id_p2=''
productos=[] #lista para productos caso n>=3
numeros = [] 
producto_recomendado=""

#Ingresar cantidad de productos simulando el carrito de compras
prod = int(input("Introduce el numero de productos a comprar: "))

#Se decide el camino que elegirá el algoritmo segun la cantidad de productos que haya
#En el caso de que solo haya un producto en el carrito
if prod==1:
    #Introduce el producto en el carrito
    cadena = input("Introduce un ID: ")
    #Se obtienen los 20 productos con mayor frecuencia de compra en conjunto con el seleccionado
    productos_max=tabla_frecuencias[cadena].nlargest(30).index
    
    #Se busca el producto con diferente categoria para realizar el cross-selling
    cat2=obtener_categoria(cat2,cadena)
    for x in range(len(productos_max)):
        cat1=obtener_categoria(cat1,productos_max[x])
        if cat1!=cat2:
            print("Producto recomendado: ",productos_max[x])
            break
#En el caso de que haya dos productos en el carrito    
elif prod==2:
    cadena = input("Introduce un ID: ")
    cadena2= input("Introduce 2do ID: ")
    tabla_frecuencias[cadena][cadena2]=tabla_frecuencias[cadena][cadena2]+1
    tabla_frecuencias[cadena2][cadena]=tabla_frecuencias[cadena2][cadena]+1
    
    #Realizar el cross-Selling para dos productos
    m1=tabla_frecuencias[cadena].max()
    m2=tabla_frecuencias[cadena2].max()

    if m1>m2:
        idmax=cadena
    else:
        idmax=cadena2

    p1_max=tabla_frecuencias[cadena].nlargest(20).index 
    p2_max=tabla_frecuencias[cadena2].nlargest(20).index

    lista_suma=[]
    for x in range(len(p1_max)):
        lista_suma.append(p1_max[x])
        lista_suma.append(p2_max[x])

    for x in range(len(lista_suma)):
        cat2=obtener_categoria(cat2,cadena)
        cat3=obtener_categoria(cat3,cadena2)
        cat1=obtener_categoria(cat1,lista_suma[x])
        if cat1!=cat2 and cat1!=cat3:
            print("Producto recomendado: ",lista_suma[x])
            break
    
#En el caso de que haya de 3 a n productos en el carrito
elif prod>=3:
    #Ingresamos los productos al carrito
    for i in range(prod):
        numero = input(f"Ingresa el número {i + 1}: ")
        numeros.append(numero)
    #Creamos una lista con id producto, cantidad, producto_alpha vacio y frecuencia
    for x in range(prod):
        productos.append([numeros[x],1,"",0])
    
    #De los productos en el carrito asignamos el producto con mayor venta en conjunto y su frecuencia
    for x in range(len(productos)):
        frec=tabla_frecuencias[productos[x][0]].max(axis=0)
        Id=tabla_frecuencias[productos[x][0]].idxmax()
        productos[x][2]=Id
        productos[x][3]=frec
    
    #Llamar funciones para la recomendación
    inicio_caso_n()

#Se realiza el aumento en 1 de la tabla de frecuencia tomando en cuenta los productos comprados en conjunto del carrito
for n1 in numeros:
    for n2 in numeros:
        if n1==n2:
            tabla_frecuencias[n1][n2]=0
        elif n1!=n2:
            tabla_frecuencias[n1][n2]=tabla_frecuencias[n1][n2]+1

#Guardar tabla frecuencias según relación:
if categorys==3:
    print("Guardar crosstab Oro")
    
elif categorys==2:
    print("Guardar crosstab plata")

elif categorys==1:
    print("Guardar crosstab Bronce")
            

Introduce el numero de productos a comprar: 1
Introduce un ID: 673
Producto recomendado:  CIG500001
Guardar crosstab Bronce
